# 4.11 Exploring a Maze

![image](http://interactivepython.org/runestone/static/pythonds/_images/maze.png)

### Here is the procedure:
• From our starting position we will first try going North one square and then recursively try our procedure from there.

• If we are not successful by trying a Northern path as the first step then we will take a step to the South and recursively repeat our procedure.

• If South does not work then we will try a step to the West as our first step and recursively apply our procedure.

• If North, South, and West have not been successful then apply the procedure recursively from a position one step to our East.

• If none of these directions works then there is no way to get out of the maze and we fail.

-> "북, 남, 서, 동" 순차적으로 이동가능하고 4가지 방향 모두 이동 불가능하면 실패




### In this algorithm, there are four base cases to consider:
1.The turtle has run into a wall. Since the square is occupied by a wall no further exploration can take place.

2.The turtle has found a square that has already been explored. We do not want to continue exploring from this position or we will get into a loop.

3.We have found an outside edge, not occupied by a wall. In other words we have found an exit from the maze.

4.We have explored a square unsuccessfully in all four directions.

-> 이미 지나간 사각형은 지나갈 수 없음, 모든 방향이 막혀있으면 실패, 막히지 않은 가장자리 발견시 성공

### algorithm:
•__init__: Reads in a data file representing a maze, initializes the internal representation of the maze, and finds the starting position for the turtle. -> maze 불러오기&표현, 시작위치 설정

•drawMaze: Draws the maze in a window on the screen. -> maze 그리기

•updatePosition: Updates the internal representation of the maze and changes the position of the turtle in the window. -> maze&위치 변화

•isExit: Checks to see if the current position is an exit from the maze. -> 탈출여부 확인

In [14]:
import turtle

PART_OF_PATH = 'O'
TRIED = '.'
OBSTACLE = '+'
DEAD_END = '-'

In [23]:
class Maze:
    #The init method takes the name of a file as its only parameter. 
    #This file is a text file that represents a maze by using “+” characters for walls, spaces for open squares, and the letter “S” to indicate the starting position.
    def __init__(self,mazeFileName):
        rowsInMaze = 0
        columnsInMaze = 0
        self.mazelist = []
        mazeFile = open(mazeFileName,'r')
        rowsInMaze = 0
        for line in mazeFile:
            rowList = []
            col = 0
            for ch in line[:-1]:
                rowList.append(ch)
                if ch == 'S':
                    self.startRow = rowsInMaze
                    self.startCol = col
                col = col + 1
            rowsInMaze = rowsInMaze + 1
            self.mazelist.append(rowList)
            columnsInMaze = len(rowList)

        self.rowsInMaze = rowsInMaze
        self.columnsInMaze = columnsInMaze
        self.xTranslate = -columnsInMaze/2
        self.yTranslate = rowsInMaze/2
        self.t = turtle.Turtle()
        self.t.shape('turtle')
        self.wn = turtle.Screen()
        self.wn.setworldcoordinates(-(columnsInMaze-1)/2-.5,-(rowsInMaze-1)/2-.5,(columnsInMaze-1)/2+.5,(rowsInMaze-1)/2+.5) #좌표범위 설정

        
    #장애물을 주황색으로 칠해주는 함수
    def drawMaze(self):
        self.t.speed(10)
        self.wn.tracer(0)
        for y in range(self.rowsInMaze):
            for x in range(self.columnsInMaze):
                if self.mazelist[y][x] == OBSTACLE:
                    self.drawCenteredBox(x+self.xTranslate,-y+self.yTranslate,'orange')
        self.t.color('black')
        self.t.fillcolor('blue')
        self.wn.update()
        self.wn.tracer(1)

        
    #(x,y)를 중심점으로 하는 사각형을 color로 색칠하는 함수
    def drawCenteredBox(self,x,y,color):
        self.t.up()
        self.t.goto(x-.5,y-.5)
        self.t.color(color)
        self.t.fillcolor(color)
        self.t.setheading(90)
        self.t.down()
        self.t.begin_fill()
        for i in range(4):
            self.t.forward(1)
            self.t.right(90)
        self.t.end_fill()

        
    #(x,y)에서 (x+self.xTranslate,-y+self.yTranslate)로 움직이는 함수
    def moveTurtle(self,x,y):
        self.t.up()
        self.t.setheading(self.t.towards(x+self.xTranslate,-y+self.yTranslate)) #(x+self.xTranslate,-y+self.yTranslate)를 향하게 방향설정
        self.t.goto(x+self.xTranslate,-y+self.yTranslate)

        
    #color 색의 사이즈가 10인 점그리는 함수   
    def dropBreadcrumb(self,color):
        self.t.dot(10,color)

        
    #(col,row)로 옮기고 PART_OF_PATH = 'green', OBSTACLE = 'red', TRIED = 'black', DEAD_END = 'red'으로 점그리는 함수
    def updatePosition(self,row,col,val=None):
        if val:
            self.mazelist[row][col] = val
        self.moveTurtle(col,row)

        if val == PART_OF_PATH:
            color = 'green'
        elif val == OBSTACLE:
            color = 'red'
        elif val == TRIED:
            color = 'black'
        elif val == DEAD_END:
            color = 'red'
        else:
            color = None

        if color:
            self.dropBreadcrumb(color)

            
    # 가장자리 위치시 TRUE를 반환하는 함수     
    def isExit(self,row,col):
        return (row == 0 or
                row == self.rowsInMaze-1 or
                col == 0 or
                col == self.columnsInMaze-1 )

    
    #idx에 해당하는 row를 반환하는 함수
    def __getitem__(self,idx):
        return self.mazelist[idx]

In [3]:
def searchFrom(maze, startRow, startColumn):
    # try each of four directions from this point until we find a way out.
    # base Case return values:
    #  1. We have run into an obstacle, return false
    maze.updatePosition(startRow, startColumn)
    if maze[startRow][startColumn] == OBSTACLE :
        return False
    #  2. We have found a square that has already been explored
    if maze[startRow][startColumn] == TRIED or maze[startRow][startColumn] == DEAD_END:
        return False
    # 3. We have found an outside edge not occupied by an obstacle
    if maze.isExit(startRow,startColumn):
        maze.updatePosition(startRow, startColumn, PART_OF_PATH)
        return True
    maze.updatePosition(startRow, startColumn, TRIED)
    # Otherwise, use logical short circuiting to try each direction
    # in turn (if needed)
    found = searchFrom(maze, startRow-1, startColumn) or \
            searchFrom(maze, startRow+1, startColumn) or \
            searchFrom(maze, startRow, startColumn-1) or \
            searchFrom(maze, startRow, startColumn+1)
    if found:
        maze.updatePosition(startRow, startColumn, PART_OF_PATH)
    else:
        maze.updatePosition(startRow, startColumn, DEAD_END)
    return found

In [4]:
myMaze = Maze('maze2.txt')
myMaze.drawMaze()
myMaze.updatePosition(myMaze.startRow,myMaze.startCol)

In [5]:
searchFrom(myMaze, myMaze.startRow, myMaze.startCol)

True

# 4.12 Dynamic Programming

### A classic example of an optimization problem involves making change using the fewest coins. 
numCoins = min{1+numCoins(originalamount-1),
               1+numCoins(originalamount-5),
               1+numCoins(originalamount-10),
               1+numCoins(originalamount-25)

In [15]:
def recMC(coinValueList,change):
    minCoins = change
    if change in coinValueList:
        return 1
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recMC(coinValueList,change-i)
            if numCoins < minCoins: #작은값으로 바꿔주기
                minCoins = numCoins
    return minCoins

print(recMC([1,5,10,25],63))

6


The trouble with the algorithm in Listing 7 is that it is extremely inefficient. The key to cutting down on the amount of work we do is to remember some of the past results so we can avoid recomputing results we already know. 

In [16]:
def recDC(coinValueList,change,knownResults): #knownResults:결과 저장할 공간 
    minCoins = change
    if change in coinValueList:
        knownResults[change] = 1
        return 1
    elif knownResults[change] > 0: #결과 존재시 반환
        return knownResults[change]
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recDC(coinValueList, change-i, knownResults)
            if numCoins < minCoins:
                minCoins = numCoins
                knownResults[change] = minCoins
    return minCoins

print(recDC([1,5,10,25],63,[0]*64))

6


![](http://interactivepython.org/runestone/static/pythonds/_images/changeTable.png)

![image](http://interactivepython.org/runestone/static/pythonds/_images/elevenCents.png)

Listing 8 is a dynamic programming algorithm to solve our change-making problem. 

In [19]:
def dpMakeChange(coinValueList,change,minCoins):
    for cents in range(change+1): #cents: 1~change
        coinCount = cents
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents-j] + 1 < coinCount:
                coinCount = minCoins[cents-j]+1
        minCoins[cents] = coinCount
    return minCoins[change]

print(dpMakeChange([1,5,10,25],63,[0]*64))

6


ActiveCode 2 shows the dpMakeChange algorithm modified to keep track of the coins used, along with a function printCoins that walks backward through the table to print out the value of each coin used. 

In [20]:
def dpMakeChange(coinValueList,change,minCoins,coinsUsed):
    for cents in range(change+1):
        coinCount = cents
        newCoin = 1
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents-j] + 1 < coinCount:
                coinCount = minCoins[cents-j]+1
                newCoin = j
        minCoins[cents] = coinCount
        coinsUsed[cents] = newCoin
    return minCoins[change]

In [21]:
def printCoins(coinsUsed,change):
    coin = change
    while coin > 0:
        thisCoin = coinsUsed[coin]
        print(thisCoin)
        coin = coin - thisCoin

In [22]:
def main():
    amnt = 63
    clist = [1,5,10,21,25]
    coinsUsed = [0]*(amnt+1)
    coinCount = [0]*(amnt+1)

    print("Making change for",amnt,"requires")
    print(dpMakeChange(clist,amnt,coinCount,coinsUsed),"coins")
    print("They are:")
    printCoins(coinsUsed,amnt)
    print("The used list is as follows:")
    print(coinsUsed)

main()

Making change for 63 requires
3 coins
They are:
21
21
21
The used list is as follows:
[1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 21, 1, 1, 1, 25, 1, 1, 1, 1, 5, 10, 1, 1, 1, 10, 1, 1, 1, 1, 5, 10, 21, 1, 1, 10, 21, 1, 1, 1, 25, 1, 10, 1, 1, 5, 10, 1, 1, 1, 10, 1, 10, 21]
